In [1]:
import csv
import copy
import dash_bootstrap_components as dbc
import dash_cytoscape as cyto
import graph_tool as gt
import graph_tool.util as gu
import os
import io
import base64
import pandas as pd
import time

from dash import Dash, dcc, html, Input, Output, State, ctx
from graph_tool import GraphView
from jupyter_dash import JupyterDash

In [2]:
def parse_file_to_network(df, g, sample):
    edge_d = df.values.tolist()
    nodes = []
    edges= []
    for edge in edge_d:
        re = edge[0]
        ge = edge[1]
        v_list = gu.find_vertex(g, g.vp.name, re)
        v = v_list[0]
        if g.vp.name[v].startswith("E") and sample != "score" and sample != "p_value":
            nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v], 'start': str(g.vp.start[v]), 'end': str(g.vp.end[v]), 'expr': str(g.vp[sample][v]), 'chr': g.vp.chr[v]}, 'classes':'black'})
        else: nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v], 'start': str(g.vp.start[v]), 'end': str(g.vp.end[v]), 'chr': g.vp.chr[v]}, 'classes':'black'})
        for e in v.all_edges():
            v2 = e.target()
            if g.vp.name[v2] == ge:
                if g.edge_properties[sample][e] >= 0: color = "green"
                else: color = "red"
                edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': str(g.edge_properties[sample][e]), 'rem': g.ep.rem[e]}, 'style':{'width':abs(g.edge_properties[sample][e])*5, 'line-color': color}})
                if g.vp.name[v2].startswith("E") and sample != "score" and sample != "p_value":
                    nodes.append({'data': {'id': g.vp.name[v2], 'label': g.vp.name[v2], 'start': str(g.vp.start[v2]), 'end': str(g.vp.end[v2]), 'expr': str(g.vp[sample][v2]), 'chr': g.vp.chr[v2]}, 'classes':'DodgerBlue'})
                else:
                    nodes.append({'data': {'id': g.vp.name[v2], 'label': g.vp.name[v2], 'start': str(g.vp.start[v2]), 'end': str(g.vp.end[v2]), 'chr': g.vp.chr[v2]}, 'classes': 'DodgerBlue'})
    cyto_list = edges+nodes
    return cyto_list

In [3]:
def get_network(node, sample, fileupload = False, df = None):
    global gr                                                       # multiple users, maybe use pickle.dump and load as solution if it fails
    g = gr
    if fileupload is True:
        return parse_file_to_network(df, g, sample)
    v_list = gu.find_vertex(g, g.vp.name, node)
    if len(v_list) != 0:
        v = v_list[0]
    else:
        ed = gu.find_edge(g, g.ep.rem, node)[0]
        v = ed.source()

    nodes = []
    edges= []
    if g.vp.name[v].startswith("E") and sample != "score" and sample != "p_value":
        nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v], 'start': str(g.vp.start[v]), 'end': str(g.vp.end[v]), 'expr': str(g.vp[sample][v]), 'chr': g.vp.chr[v]}, 'classes':'black'})
    else:
        nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v], 'start': str(g.vp.start[v]), 'end': str(g.vp.end[v]), 'chr': g.vp.chr[v]}, 'classes':'black'})
    for e, w in zip(v.all_edges(), v.all_neighbors()):
            if g.edge_properties[sample][e] >= 0: color = "green"
            else: color = "red"
            edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': str(g.edge_properties[sample][e]), 'rem': g.ep.rem[e]}, 'style':{'width':abs(g.edge_properties[sample][e])*5, 'line-color': color}})
            if g.vp.name[w].startswith("E") and sample != "score" and sample != "p_value":
                nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w], 'start': str(g.vp.start[w]), 'end': str(g.vp.end[w]), 'expr': str(g.vp[sample][w]), 'chr': g.vp.chr[w]}, 'classes':'DodgerBlue'})
            else:
                nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w], 'start': str(g.vp.start[w]), 'end': str(g.vp.end[w]), 'chr': g.vp.chr[w]}, 'classes': 'DodgerBlue'})
    cyto_list = edges+nodes
    return cyto_list

In [4]:
def filter_network(graph, weight_filter_up, weight_filter_low, checklist_show):
    netw = copy.deepcopy(graph)
    filt_net = []
    filt_nodes = set()
    if netw is not None:
        for e in netw:
            if e.get('data').get('source') is not None:
                if not ((weight_filter_up != None and float(e.get('data').get('label')) > weight_filter_up) or (weight_filter_low != None and float(e.get('data').get('label')) < weight_filter_low)):
                    if 'Show weights' not in checklist_show:
                        e.get('data').pop('label')
                        filt_net.append(e)
                    else:
                        filt_net.append(e)
                    filt_nodes.add(e.get('data').get('source'))
                    filt_nodes.add(e.get('data').get('target'))
            elif e.get('data').get('id') in filt_nodes:
                filt_net.append(e)
    return filt_net

In [5]:
celltype_name = {}
with open("../data/sampleToCelltype.csv") as celltype_file:
    celltype_name["allCelltypeGraph"] = "all cell types"
    for line in celltype_file:
        if not line.startswith("sample"):
            l = line.split(",")
            celltype_name[l[1]] = l[2].replace("\n", "")
celltype_file.close()

In [6]:
modal_general = dbc.Modal([
                        dbc.ModalHeader(dbc.ModalTitle(html.Strong("Documentation - REMnet"))),
                        dbc.ModalBody(html.P(children=(html.Strong("REMnet"), " is an interactive network application. It allows the user to inspect gen regulatory networks from the EpiRegio database or entering a costum network.", html.Br(),
                                                       "The app is separated in three parts. On the left you can the data you want to show or can upload a file to show a customized graph.",html.Br(),
                                                       "In the middle the respective graph is shown and can be moved. Edges thickness and color are adjusted to the edge weight.", html.Br(),
                                                       "On the right general info of the clicked node is shown. You can use the filters to adjust the nodes and edges shown based on several attributes.", html.Br(), html.Br(),
                                                       html.Strong("Statistics:"),html.Br(),
                                                       "Number of nodes: 1.541.001.", html.Br(),
                                                       "Number of edges: 2.404.861.", html.Br(),
                                                       "Number of genes: 35.379", html.Br(),
                                                       "Number of REMs: 1.140.336", html.Br(),
                                                       "Number of CREMs: 365.286"
                                                       ))),
                        ],
                        id="modal-general",
                        is_open=False,
                        size="lg",)

modal_select = dbc.Modal([
                        dbc.ModalHeader(dbc.ModalTitle(html.Strong("Documentation - Select"))),
                        dbc.ModalBody(html.P(children=(html.Strong("Select cell type:"),
                                                       html.Li("specific cell types or a general network of cell types are available"), html.Br(),
                                                       html.Strong("Select sample:"),
                                                       html.Li("sample names for the respective cell types"),
                                                       html.Li("p-value or score for general network of all cell types"),
                                                       html.Li("show the respective edge weights"), html.Br(),
                                                       html.Strong("Select gene or enter CREM/REM:"),
                                                       html.Li("show selected node as seed node with all neighboring edges and nodes"),
                                                       html.Li("for CREM/REM enter ID in described range"),html.Br(),
                                                       html.Strong("File upload:"),
                                                       html.Li("upload custmized network file to display"),
                                                       html.Li("only .csv files accepted"),
                                                       html.Li("file format: 'REM0000001,ENSG0000001' with 'REM,GENE' as header"),
                                                       ))),
                        ],
                        id="modal-select",
                        is_open=False,
                        size="lg",)

modal_filter = dbc.Modal([
                        dbc.ModalHeader(dbc.ModalTitle(html.Strong("Documentation - Filter"))),
                        dbc.ModalBody(html.P(children=(html.Strong("Gene degree filter:"),
                                                       html.Li("select gene dropdown filtered on entered degree"),
                                                       html.Li("genes with larger degree than input are filtered"),html.Br(),
                                                       html.Strong("Upper weight filter:"),
                                                       html.Li("edges with smaller edge weight than input are filtered"),html.Br(),
                                                       html.Strong("Lower weight filter:"),
                                                       html.Li("edges with larger edge weight than input are filtered"),html.Br(),
                                                       html.Strong("Show weights:"),
                                                       html.Li("select to show or hide edge weights"),html.Br(),
                                                       ))),
                        ],
                        id="modal-filter",
                        is_open=False,
                        size="lg",)

In [7]:
app = Dash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = html.Div([
    html.Div([
        html.H1("REMnet", style={"textAlign":"center", "fontSize":40, 'font-family':'Consolas'})
    ]),

    html.Div([
        html.Div([
            html.Div([
                dbc.Button("Documentation", id="open-general", n_clicks=0, style={'borderRadius': '5px'}),
                modal_general,
            ], style={'margin-bottom': '10px'}),
            html.Div([
                html.Div([
                    html.Div([
                        "Select cell type",
                        dcc.Dropdown(
                            id='dropdown-graph',
                            value=os.listdir('../data/graph-tool/')[0][:-3],
                            clearable=False,
                            options=[
                                {'label': celltype_name.get(name[:-3]), 'value': name[:-3]}
                                for name in os.listdir('../data/graph-tool/')
                            ]
                        )
                    ], style={'display': 'block', 'margin-bottom': '10px'}
                    ),
                    html.Div([
                        "Select sample",
                        dcc.Dropdown(
                            id= 'dropdown-sample',
                            value= '',
                            clearable=False,
                            options=[]
                        )
                    ], style={'display': 'block', 'margin-bottom': '10px'}),

                    html.Div([
                        "Select gene",
                        dcc.Dropdown(
                        id='dropdown-gen',
                        clearable=False,
                        options=[]
                        )
                    ],style={'display': 'block', 'margin-bottom': '10px'}
                    ),
                    html.Div(['Enter CREM ID from 1 to 365.286: ',
                        dcc.Input(id='input-crem', type='number', value = 1, step=1, min=1, max=365286, debounce=True),
                    ], style={'margin-bottom': '10px'}
                    ),
                    html.Div(['Enter REM ID from 1 to 2.404.862: ',
                        dcc.Input(id='input-rem', type='number', value = 1, step=1, min=1, max=2404862, debounce=True),
                    ], style={'margin-bottom': '10px'}
                    ),
                    html.Div([
                        dcc.Upload(id='upload-file', children=html.Div(['Drag and Drop or ', html.A('Select a File')]),style={'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px', 'textAlign': 'center','margin-bottom': '10px'},),
                        html.Div(id='output-upload-file'),
                    ], style={'margin-bottom': '10px'}
                    ),
                    html.Div([
                        dbc.Button(html.Strong("?"), id="open-select", n_clicks=0, style={'borderRadius': '5px'}),
                        modal_select,
                    ]),
                ], style={'display': 'inline-block', 'padding' : '15px 15px 15px 15px'}),
            ], style={"border":"1px rgba(0,0,0,.125) solid", 'borderRadius':'5px', 'margin-bottom': '10px'}),

        ], style={'display': 'inline-block', 'vertical-align': 'top', 'width': '14%'}
        ),
        html.Div([
            dcc.Loading(children=[html.P(id="loading")], type="circle"),
            dcc.Loading(
                children=[
                    html.Div([
                cyto.Cytoscape(
                    id='network-graph',
                    style={'width': '100%', 'height': '700px'},
                    layout={'name': 'concentric', 'avoidOverlap': 'true', 'nodeDimensionsIncludeLabels': 'true'},
                    elements=[],
                    stylesheet=[
            #             # Group selectors
                        {
                            'selector': 'node',
                            'style': {
                                'content': 'data(label)'
                            }
                        },
                        {
                            'selector': 'edge',
                            'style': {
                                'curve-style': 'bezier',
                                'content': 'data(label)'
                            }
                        },
                        {
                            'selector': '.DodgerBlue',
                            'style': {
                                'background-color': 'DodgerBlue',
                                'line-color': 'DodgerBlue'
                            }
                        },
                        {
                            'selector': '.black',
                            'style': {
                                'background-color': 'black',
                                'line-color': 'black'
                            }
                        },
                        {
                            'selector': '.green',
                            'style': {
                                'background-color': 'green',
                                'line-color': 'green'
                            }
                        },
                        {
                            'selector': '.red',
                            'style': {
                                'background-color': 'red',
                                'line-color': 'red'
                            }
                        }
                    ]
                ),
            ]),
                ],type='circle',
            ),
        ], style={ 'vertical-align': 'top', 'display': 'inline-block', 'width': '71.5%', 'height': '1000px'}
        ),
        html.Div([
            html.Div([
                html.Div([
                    html.Strong('Node info'),
                    html.P(id= 'mouse-click'),
                ], style={'display': 'inline-block', 'padding' : '15px 15px 2px 15px'}),
            ], style={"border":"0.5px rgba(0,0,0,.125) solid", 'borderRadius':'5px', 'margin-bottom': '10px'}),
            html.Div([
                html.Div([
                    html.Div([
                        html.P("Gene degree filter: "),
                        dcc.Slider(id='filter-gen', min=0, max=108, step=1, value=0, marks=None, tooltip={"placement": "bottom", "always_visible": True}),
                    ], style={'margin-bottom': '10px'}),

                    html.Div([
                        "Upper bond weight filter:  ",
                        dcc.Input(id='filter-weight-up', type='number', debounce=True),
                        html.Div(id= 'filter-text-up')
                    ], style={'margin-bottom': '10px'}),
                    html.Div([
                        "Lower bond weight filter:  ",
                        dcc.Input(id='filter-weight-low', type='number', debounce=True),
                        html.Div(id= 'filter-text-low')
                    ], style={'margin-bottom': '10px'}),
                    dcc.Checklist(options=['Show weights'], id='checklist-show', value=['Show weights'], style={'margin-bottom':'10px'}),
                    html.Div([
                        dbc.Button(html.Strong("?"), id="open-filter", n_clicks=0, style={'borderRadius': '5px'}),
                        modal_filter,
                    ]),

                ], style={'display': 'inline-block', 'padding' : '15px 15px 15px 15px'}),
            ], style={"border":"0.5px rgba(0,0,0,.125) solid", 'borderRadius':'5px', 'margin-bottom': '10px'}),

            dbc.Button('Reset filter', id= 'reset-filter', n_clicks=0, outline=True, color="primary", className="me-1", style={'borderRadius': '5px'}),

        ], style={'vertical-align': 'top', 'display': 'inline-block', 'width': '14%'}),

    ]),

    dcc.Store(id='loaded-graph'),
    dcc.Store(id='gen-degrees'),

], style={'margin-left' : '10px', 'margin-right' : '10px', 'margin-top' : '10px', 'margin-bottom' : '10px'})

@app.callback(
    Output("modal-select", "is_open"),
    Input("open-select", "n_clicks"),
    State("modal-select", "is_open"),
)
def toggle_modal_select(select, is_open):
    if select:
        return not is_open
    return is_open

@app.callback(
    Output("modal-filter", "is_open"),
    Input("open-filter", "n_clicks"),
    State("modal-filter", "is_open"),
)
def toggle_modal_filter(filter, is_open):
    if filter:
        return not is_open
    return is_open

@app.callback(
    Output("modal-general", "is_open"),
    Input("open-general", "n_clicks"),
    State("modal-general", "is_open"),
)
def toggle_modal_general(general, is_open):
    if general:
        return not is_open
    return is_open

@app.callback(Output('dropdown-sample', 'options'),                     # disable sample dropdown if allCelltypeGraph or only one sample in celltype
              Output('dropdown-sample' ,'value'),
              Output('gen-degrees', 'data'),
              Output('loading', "children"),
              Input('dropdown-graph', 'value'))
def update_file(celltype):
    global gr                                                       # multiple users, maybe use pickle.dump and load as solution if it fails
    gr = gt.load_graph(f"../data/graph-tool/{celltype}.gt")
    gen_degrees = {gr.vp.name[v] : len(gr.get_in_edges(v)) for v in gr.vertices() if gr.vp.name[v].startswith("E")}
    sample_list = [s for s in gr.edge_properties if s not in ("celltype", "celltypeID", "rem")]
    val_sample = sample_list[0]
    return sample_list, val_sample, gen_degrees, ""

@app.callback(Output('dropdown-gen', 'options'),            # dauert zu lange, vllt sideeffect, aber eig überprüft dass kein sideeffect
              Output('dropdown-gen', 'value'),
              Output('loading', "children", allow_duplicate=True),

              Input('dropdown-graph', 'value'),
              Input('filter-gen', 'value'),
              Input('gen-degrees', 'data'),
              prevent_initial_call=True)
def update_gen_dropdown(celltype, gen_filter, gen_degrees):
    if ctx.triggered_id == 'dropdown-graph':
        gen_list = list(gen_degrees.keys())
        val_gen = gen_list[0]
        return gen_list, val_gen, ""
    elif ctx.triggered_id == 'filter-gen' and gen_filter is not None:
        gen_list = [k for k, v in gen_degrees.items() if v >= gen_filter]
        return gen_list, gen_list[0], ""
    else:
        gen_list = list(gen_degrees.keys())
        val_gen = gen_list[0]
        return gen_list, val_gen, ""


@app.callback(Output('network-graph', 'elements'),
              Output('loaded-graph', 'data'),
              Output('output-upload-file', 'children'),
              Output('filter-weight-up', 'value'),
              Output('filter-weight-low', 'value'),
              Output('checklist-show', 'value'),
              Output('upload-file', 'contents'),
              Output('upload-file', 'filename'),

              Input('dropdown-gen', 'value'),
              Input('input-rem', 'value'),
              Input('input-crem', 'value'),
              Input('dropdown-sample', 'value'),
              Input('upload-file', 'contents'),

              State('network-graph', 'elements'),
              State('upload-file', 'filename'),
              State('output-upload-file', 'children'))
def update_elements(gen, rem_int, crem_int, sample, contents, elements, filename, out_file):
    if ctx.triggered_id == 'upload-file':
        if contents is not None and '.csv' in filename:
            content_type, content_string = contents.split(',')
            decoded = base64.b64decode(content_string)
            try:
                df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
                net = get_network(gen, sample, True, df)
                return net, net, html.Div(['File uploaded successfully.']), None, None, ["Show weights"], None, None
            except Exception as e:
                return elements, elements, html.Div([str(e)]), None, None, ["Show weights"], None, None
        else: return elements, elements, html.Div(["Please provide a csv file in the correct format."]), None, None, ["Show weights"], None, None
    if ctx.triggered_id == 'dropdown-gen':
        net = get_network(gen, sample)
        return net, net, None, None, None, ["Show weights"], None, None
    elif ctx.triggered_id == 'input-rem':
        rem = f'REM{rem_int:>07}'
        net = get_network(rem, sample)
        return net, net, None, None, None, ["Show weights"], None, None
    elif ctx.triggered_id == 'input-crem':
        crem = f'CREM{crem_int:>07}'
        net = get_network(crem, sample)
        return net, net, None, None, None, ["Show weights"], None, None
    else:
        net = get_network(gen, sample)
        return net, net, None, None, None, ["Show weights"], None, None

@app.callback(Output('network-graph', 'elements', allow_duplicate=True),
              Input('filter-weight-up', 'value'),
              Input('filter-weight-low', 'value'),
              Input('checklist-show', 'value'),
              State('network-graph', 'elements'),
              State('loaded-graph', 'data'),
              prevent_initial_call=True)
def filter_elements(filter_up, filter_low, show, elements, loaded_graph):
    return filter_network(loaded_graph, filter_up, filter_low, show)

@app.callback(Output('filter-weight-up', 'value', allow_duplicate=True),
              Output('filter-weight-low', 'value', allow_duplicate=True),
              Output('filter-gen', 'value'),
              Input('reset-filter', 'n_clicks'),
              prevent_initial_call=True)
def reset_filter(n_clicks):
    return None, None, None


@app.callback(Output('mouse-click', 'children'),
              Input('network-graph', 'tapNodeData'))
def node_hover(data):
    if data:
        if len(data) == 6:
            return f'Name: {data["label"]}', html.Br(), f'Expression: {data["expr"]}', html.Br(), f'Chromosome: {data["chr"]}', html.Br(),f'Start: {data["start"]} ', html.Br(),f'End: {data["end"]}'
        else:
            return f'Name: {data["label"]} ', html.Br(), f'Chromosome: {data["chr"]}', html.Br(),f' Start: {data["start"]} ', html.Br(),f' End: {data["end"]}'
    else:
        return "Click Node to see info"


# app.run_server(port=8000, host='127.0.0.1', debug=False, jupyter_mode="external")
app.run(debug=True, jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>